# Lesson 6: Hooks & Structured Output

This interactive notebook teaches you how to extend agent functionality and extract type-safe data:

- ✅ Lifecycle hooks (BeforeInvocationEvent, AfterInvocationEvent)
- ✅ Tool execution hooks (BeforeToolCallEvent, AfterToolCallEvent)
- ✅ HookProvider pattern for composable extensions
- ✅ Structured output with Pydantic models
- ✅ Type-safe data extraction from conversations

**Estimated time:** 4 hours

**What you'll build:** Request loggers, tool monitors, and type-safe data extractors!

## Setup

Import necessary modules and configure the environment:

In [ ]:
from lesson_utils import load_environment, create_working_model, check_api_keys
from strands import Agent, tool
from strands.hooks import HookProvider, HookRegistry
from strands.hooks.events import (
    BeforeInvocationEvent,
    AfterInvocationEvent,
    BeforeToolCallEvent,
    AfterToolCallEvent,
    MessageAddedEvent,
)
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime

# Load environment and check API keys
load_environment()
check_api_keys()

print("🎯 Lesson 6: Hooks & Structured Output")
print("=" * 50)

## Part 1: Basic Lifecycle Hooks

Hooks are a composable extensibility mechanism for extending agent functionality by subscribing to events throughout the agent lifecycle.

There are two ways to register hooks:
1. **Individual callback registration** - Simple and direct
2. **HookProvider pattern** - Composable and reusable (recommended)

### Method 1: Individual Callback Registration

In [ ]:
model = create_working_model()

if model:
    agent = Agent(model=model)

    def log_request_start(event: BeforeInvocationEvent) -> None:
        """Called before agent processes a request."""
        print(f"   🚀 Request started for agent: {event.agent.name}")
        print(f"      Timestamp: {datetime.now().strftime('%H:%M:%S')}")

    def log_request_end(event: AfterInvocationEvent) -> None:
        """Called after agent completes a request."""
        print(f"   ✅ Request completed for agent: {event.agent.name}")
        print(f"      Timestamp: {datetime.now().strftime('%H:%M:%S')}")

    # Register individual callbacks
    agent.hooks.add_callback(BeforeInvocationEvent, log_request_start)
    agent.hooks.add_callback(AfterInvocationEvent, log_request_end)

    response = agent("What is 2 + 2?")
    print(f"\n   Agent response: {response}")
else:
    print("⚠️ No API key available")

### Method 2: HookProvider Pattern (Recommended)

The HookProvider protocol allows a single object to register callbacks for multiple events. This is the preferred composable approach:

In [ ]:
model = create_working_model()

if model:
    class RequestLoggingHook(HookProvider):
        """Reusable hook for logging request lifecycle."""

        def register_hooks(self, registry: HookRegistry) -> None:
            """Register all callbacks for this hook."""
            registry.add_callback(BeforeInvocationEvent, self.log_start)
            registry.add_callback(AfterInvocationEvent, self.log_end)
            registry.add_callback(MessageAddedEvent, self.log_message)

        def log_start(self, event: BeforeInvocationEvent) -> None:
            print(f"   📝 [Hook] Request starting...")

        def log_end(self, event: AfterInvocationEvent) -> None:
            print(f"   📝 [Hook] Request completed!")

        def log_message(self, event: MessageAddedEvent) -> None:
            """Log when messages are added to conversation history."""
            role = event.message.get("role", "unknown")
            print(f"   📝 [Hook] Message added: role={role}")

    # Pass hook provider during agent initialization
    agent_with_hooks = Agent(
        model=model,
        hooks=[RequestLoggingHook()]
    )

    response = agent_with_hooks("Tell me a fun fact about Python programming.")
    print(f"\n   Agent response: {response}")
else:
    print("⚠️ No API key available")

## Part 2: Tool Execution Hooks

Hooks can intercept and modify tool execution:
- Log tool usage and parameters
- Modify tool inputs before execution
- Process or format tool results after execution
- Access invocation_state for context-aware behavior

Let's create a calculator tool and monitor it with hooks:

In [ ]:
@tool
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression and return the result."""
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

model = create_working_model()

if model:
    class ToolMonitoringHook(HookProvider):
        """Hook for monitoring and modifying tool execution."""

        def register_hooks(self, registry: HookRegistry) -> None:
            registry.add_callback(BeforeToolCallEvent, self.log_tool_call)
            registry.add_callback(AfterToolCallEvent, self.format_result)

        def log_tool_call(self, event: BeforeToolCallEvent) -> None:
            """Log tool calls before execution."""
            tool_name = event.tool_use.get("name", "unknown")
            tool_input = event.tool_use.get("input", {})

            print(f"   🔧 Tool call: {tool_name}")
            print(f"      Input: {tool_input}")

            # Access invocation_state for context
            user_id = event.invocation_state.get("user_id", "anonymous")
            print(f"      User: {user_id}")

        def format_result(self, event: AfterToolCallEvent) -> None:
            """Format tool results after execution."""
            tool_name = event.tool_use.get("name", "unknown")

            if tool_name == "calculator":
                # Modify the tool result to add formatting
                original_content = event.result["content"][0]["text"]
                event.result["content"][0]["text"] = f"Calculation result: {original_content}"
                print(f"   ✨ Formatted result for {tool_name}")

    # Create agent with tool monitoring hook
    agent = Agent(
        model=model,
        tools=[calculator],
        hooks=[ToolMonitoringHook()]
    )

    # Execute with invocation_state for context
    response = agent(
        "What is 15 * 7?",
        user_id="user123"  # Custom context passed to hooks
    )

    print(f"\n   Agent response: {response}")
else:
    print("⚠️ No API key available")

## Part 3: Structured Output with Pydantic

Structured output enables type-safe, validated responses using Pydantic models. Instead of parsing raw text, you define the exact structure and receive a validated Python object.

### Example 1: Basic Structured Extraction

In [ ]:
model = create_working_model()

if model:
    class PersonInfo(BaseModel):
        """Extract person information from text."""
        name: str = Field(description="Full name of the person")
        age: int = Field(description="Age in years")
        occupation: str = Field(description="Job or profession")

    agent = Agent(model=model)

    # Extract structured information from text
    result = agent.structured_output(
        PersonInfo,
        "Sarah Chen is a 28-year-old data scientist working at a tech startup."
    )

    print(f"   Name: {result.name}")
    print(f"   Age: {result.age}")
    print(f"   Occupation: {result.occupation}")
    print(f"   Type: {type(result)}")
else:
    print("⚠️ No API key available")

### Example 2: Complex Nested Models

Pydantic supports sophisticated nested data structures:

In [ ]:
model = create_working_model()

if model:
    class Address(BaseModel):
        """Nested model for address information."""
        street: str
        city: str
        country: str
        postal_code: Optional[str] = None

    class Contact(BaseModel):
        """Nested model for contact information."""
        email: Optional[str] = None
        phone: Optional[str] = None

    class DetailedPerson(BaseModel):
        """Complete person profile with nested data."""
        name: str = Field(description="Full name")
        age: int = Field(description="Age in years")
        address: Address = Field(description="Home address")
        contacts: List[Contact] = Field(default_factory=list, description="Contact methods")
        skills: List[str] = Field(default_factory=list, description="Professional skills")

    agent = Agent(model=model)

    result = agent.structured_output(
        DetailedPerson,
        """
        Extract information about: John Smith, a 35-year-old software architect.
        He lives at 123 Market Street, San Francisco, CA 94103.
        Contact him at john.smith@example.com or (415) 555-0123.
        His skills include Python, Kubernetes, and system design.
        """
    )

    print(f"   Name: {result.name}")
    print(f"   Age: {result.age}")
    print(f"   Address: {result.address.street}, {result.address.city}")
    print(f"   Email: {result.contacts[0].email if result.contacts else 'N/A'}")
    print(f"   Skills: {', '.join(result.skills)}")
else:
    print("⚠️ No API key available")

### Example 3: Using Conversation History

Structured output can extract information from existing conversation context:

In [ ]:
model = create_working_model()

if model:
    class CityInfo(BaseModel):
        """Extract city information from conversation."""
        city: str
        country: str
        population: Optional[int] = None
        famous_for: List[str] = Field(default_factory=list)
        best_season: Optional[str] = None

    conversation_agent = Agent(model=model)

    # Build up conversation context
    print("Building conversation context...")
    conversation_agent("What do you know about Tokyo, Japan?")
    conversation_agent("What's the population and what is it famous for?")
    conversation_agent("When is the best time to visit?")

    # Extract structured information from the conversation
    city_data = conversation_agent.structured_output(
        CityInfo,
        "Extract all the information we discussed about this city"
    )

    print(f"\n   City: {city_data.city}")
    print(f"   Country: {city_data.country}")
    print(f"   Population: {city_data.population if city_data.population else 'Not specified'}")
    print(f"   Famous for: {', '.join(city_data.famous_for[:3])}...")  # Show first 3
    print(f"   Best season: {city_data.best_season if city_data.best_season else 'Not specified'}")
else:
    print("⚠️ No API key available")

## Part 4: Combining Hooks and Structured Output

Hooks and structured output work together for production-ready patterns like auditing and logging:

In [ ]:
model = create_working_model()

if model:
    class AuditHook(HookProvider):
        """Audit hook that tracks structured output requests."""

        def __init__(self):
            self.structured_requests = []

        def register_hooks(self, registry: HookRegistry) -> None:
            registry.add_callback(BeforeInvocationEvent, self.track_request)
            registry.add_callback(AfterInvocationEvent, self.log_completion)

        def track_request(self, event: BeforeInvocationEvent) -> None:
            print(f"   📊 Audit: Request started at {datetime.now().strftime('%H:%M:%S')}")

        def log_completion(self, event: AfterInvocationEvent) -> None:
            print(f"   📊 Audit: Request completed at {datetime.now().strftime('%H:%M:%S')}")

    class ProductInfo(BaseModel):
        """Product information model."""
        name: str
        price: float
        category: str
        in_stock: bool

    # Create agent with audit hook
    agent = Agent(
        model=model,
        hooks=[AuditHook()]
    )

    # Use structured output with hook monitoring
    product = agent.structured_output(
        ProductInfo,
        "Extract product info: The UltraBook Pro laptop costs $1299, it's a computer, and is currently available."
    )

    print(f"\n   Product: {product.name}")
    print(f"   Price: ${product.price}")
    print(f"   Category: {product.category}")
    print(f"   Available: {'Yes' if product.in_stock else 'No'}")
else:
    print("⚠️ No API key available")

## Experiments

Now it's your turn! Try these experiments:

### Exercises:
1. **Performance Monitor** - Create a hook that tracks request duration using BeforeInvocationEvent and AfterInvocationEvent
2. **Tool Cache** - Build a ToolCacheHook that caches tool results to avoid redundant calls
3. **Validation Hook** - Implement a hook that validates tool inputs before execution
4. **Meeting Extractor** - Create a Pydantic model to extract meeting details (date, time, attendees, agenda)
5. **File Logger** - Build a hook that automatically logs structured outputs to a JSON file
6. **Multi-Hook Agent** - Combine multiple hooks (logging + performance + audit) on one agent
7. **Tool Cancellation** - Experiment with `cancel_tool` in BeforeToolCallEvent to prevent execution
8. **Parameter Modifier** - Create a hook that modifies `tool_use` parameters before execution

Use the cell below for your experiments:

In [ ]:
# Your experiments here!


## ✅ Success Criteria

You've completed Lesson 6 if:

- ✅ Hooks successfully intercept lifecycle events
- ✅ BeforeInvocationEvent and AfterInvocationEvent fire correctly
- ✅ Structured output returns validated Pydantic models
- ✅ Type safety enforced (IDE autocomplete works)
- ✅ Hooks and structured output work together

## 💡 Key Concepts Learned

- **Lifecycle Hooks** - BeforeInvocationEvent, AfterInvocationEvent, MessageAddedEvent
- **Tool Hooks** - BeforeToolCallEvent, AfterToolCallEvent for tool monitoring
- **HookProvider Pattern** - Composable, reusable hook implementations
- **Structured Output** - Type-safe data extraction with Pydantic
- **invocation_state** - Passing context to hooks for aware behavior

## Next Steps

- **Lesson 7**: Advanced Tools, Context & MCP - Class-based tools and conversation management
- **Lesson 10**: Production observability with OpenTelemetry

Ready to continue? Open `lesson_07_advanced_tools.ipynb`!